In [1]:
from pyspark.sql import SparkSession, functions as f, types as t, Column, DataFrame
from pyspark.sql.window import Window, WindowSpec

spark = SparkSession.builder.getOrCreate()

# ld_file = 'gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/pics_credible_set'
# linked_data = spark.read.parquet(ld_file).persist()

# linked_data.show()
# linked_data.count()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/08 16:55:48 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/01/08 16:55:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/01/08 16:55:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/01/08 16:55:48 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [28]:
linked_data.show(1, False, True)

# # Associations with LD set.
# print(tags_sorted.select('variantId', 'studyId').distinct().count())

# # Associations 
# (
#     tags_sorted
#     .groupBy('studyId', 'pair_id')
#     .count()
#     # .groupBy('count')
#     .filter(f.col('count') > 1)
#     .show(truncate=False)
# )

-RECORD 0---------------------------------------------------------------------------------------------------------------------
 chromosome           | null                                                                                                  
 variantId            | chr1:96582508                                                                                         
 studyId              | GCST002593                                                                                            
 position             | null                                                                                                  
 referenceAllele      | null                                                                                                  
 alternateAllele      | null                                                                                                  
 pValueMantissa       | 9.0                                                                                    

In [51]:
# How many resolved lead exist:
tags_sorted = (
    linked_data
    .filter(f.col('tagVariantId').isNotNull())
    .withColumn('pair_id', f.concat_ws('-', f.array_sort(f.array(f.col('variantId'), f.col('tagVariantId')))))
    # .withColumn(
    #     'distance',
    #     f.abs(
    #         f.split(f.col('variantId'), '_').getItem(1).cast(t.IntegerType()) - 
    #         f.split(f.col('tagVariantId'), '_').getItem(1).cast(t.IntegerType())
    #     )
    # ) 
    .persist()
)

resolved_count = tags_sorted.select('studyId', 'variantId').distinct().count()
print(f'Number of resolved associations: {resolved_count}')

Number of resolved associations: 392106


In [52]:
# Associations 
(
    tags_sorted
    .groupBy('studyId', 'pair_id', 'distance')
    .count()
    .groupBy('count')
    # .filter(f.col('count') > 1)
    .count()
    .show(100, truncate=False)
)

+-----+--------+
|count|count   |
+-----+--------+
|6    |1       |
|5    |78      |
|1    |30122527|
|2    |398407  |
|4    |279     |
+-----+--------+



In [57]:
columns = [
    'studyId',
    'variantId',
    'tagVariantId', 
    'qualityControl', 
    'pValueMantissa', 
    'pValueExponent',
    'R_overall',
    'pics_99_perc_credset'
]

problematic = (
    tags_sorted
    .withColumn(
        'count',
        f.count(f.col('pair_id')).over(Window.partitionBy('studyId', 'pair_id'))
    )
    .filter(f.col('count') > 1)
    .select(*columns)
    .persist()
)

problematic.count()
problematic.show(1, False, True)

-RECORD 0-------------------------------
 studyId              | GCST000248      
 variantId            | 10_63361805_C_A 
 tagVariantId         | 10_63515167_A_G 
 qualityControl       | []              
 pValueMantissa       | 5.0             
 pValueExponent       | -10             
 R_overall            | 0.983452        
 pics_99_perc_credset | true            
only showing top 1 row



In [60]:
problematic.select('studyId', 'variantId').distinct().count()

47317

In [48]:
(
    tags_sorted
    .filter(
        (f.col('studyId') == study_id) & 
        (f.col('pair_id') == pair_id)
    )
    .select(*columns)
    .show(truncate=False)
)

+------------+-----------------------------------+-----+
|studyId     |pair_id                            |dist |
+------------+-----------------------------------+-----+
|GCST90026657|2_233687171_A_C-2_233703808_CTCTG_C|16637|
+------------+-----------------------------------+-----+



In [47]:
'2_233687171_A_C-2_233703808_CTCTG_C'.split('_')

['2', '233687171', 'A', 'C-2', '233703808', 'CTCTG', 'C']

In [63]:
problematic.select('studyId').distinct().show(100)

+---------------+
|        studyId|
+---------------+
|   GCST90060602|
|   GCST006072_2|
|     GCST008025|
|   GCST90060831|
|   GCST90060635|
|   GCST005463_1|
|     GCST003264|
|   GCST003265_1|
|   GCST90060139|
|   GCST90060328|
|   GCST90060772|
|     GCST005149|
|     GCST004750|
|   GCST90060824|
|   GCST90019502|
|   GCST90053837|
| GCST009391_166|
| GCST011427_412|
|   GCST90060638|
|   GCST90060939|
|   GCST90026785|
|     GCST003266|
|     GCST005790|
|   GCST90132904|
|   GCST90027092|
|   GCST90060164|
|     GCST005956|
| GCST90086172_1|
|   GCST90129558|
|   GCST90027007|
|     GCST011141|
|     GCST006280|
|     GCST001693|
|   GCST90060808|
|GCST006585_1466|
|   GCST90060211|
|   GCST90060162|
|   GCST90060818|
|   GCST90026828|
|   GCST90085921|
|     GCST002306|
|   GCST90027091|
|     GCST009577|
|     GCST012200|
|   GCST90026965|
|     GCST002745|
|     GCST003243|
|   GCST90011735|
|  GCST006249_24|
|     GCST009826|
|GCST006585_1008|
|  GCST009537_20|
|     GCST

In [65]:
problematic.filter(f.size(f.col('qualityControl')) == 0).select('studyId').distinct().show(

)

+---------------+
|        studyId|
+---------------+
|  GCST009518_37|
|   GCST90060602|
|     GCST003266|
|     GCST008025|
|   GCST90060808|
|   GCST90060831|
|     GCST004750|
|     GCST009577|
|   GCST90060164|
|   GCST90019502|
|GCST006585_1379|
|   GCST90060139|
|   GCST90060772|
|   GCST90060818|
|     GCST000248|
|   GCST90026657|
|GCST006585_1729|
|   GCST90060635|
|   GCST90132904|
|   GCST90060824|
+---------------+
only showing top 20 rows



In [20]:
column_names = [
    'studyId',
    'variantId',
    'tagVariantId',
    'pValueMantissa',
    'pValueExponent',
    'R_overall',
]

data = [
    # First study, variant 1:
    ('s1', 'v1', 'v1', 1, -9, 1.0),
    ('s1', 'v1', 'v2', 1, -9, 0.98),
    ('s1', 'v1', 'v4', 1, -9, 0.98),
    # First study, variant 2, higher p-value, explained by v1:
    ('s1', 'v2', 'v1', 1, -8, 0.98),
    ('s1', 'v2', 'v2', 1, -8, 1.0),
    ('s1', 'v2', 'v4', 1, -8, 0.98),
    # First study, variant 3, not resolved in ld:
    ('s1', 'v3', None, 1, -18, None),
    # First study, variant 4: higher p-value, explained by v1
    ('s1', 'v4', 'v1', 4, -9, 0.98),
    ('s1', 'v4', 'v2', 4, -9, 0.98),
    ('s1', 'v4', 'v4', 4, -9, 1.0),
    # Second study, same variant discovered earlier:
    ('s2', 'v2', 'v2', 1, -12, 1.0),
    ('s2', 'v2', 'v1', 1, -12, 0.98),
]

df = (
    spark.createDataFrame(data, column_names)
    .withColumn('qualityControl', f.array())
    .persist()
)
df.show()

+-------+---------+------------+--------------+--------------+---------+--------------+
|studyId|variantId|tagVariantId|pValueMantissa|pValueExponent|R_overall|qualityControl|
+-------+---------+------------+--------------+--------------+---------+--------------+
|     s1|       v1|          v1|             1|            -9|      1.0|            []|
|     s1|       v1|          v2|             1|            -9|     0.98|            []|
|     s1|       v1|          v4|             1|            -9|     0.98|            []|
|     s1|       v2|          v1|             1|            -8|     0.98|            []|
|     s1|       v2|          v2|             1|            -8|      1.0|            []|
|     s1|       v2|          v4|             1|            -8|     0.98|            []|
|     s1|       v3|        null|             1|           -18|     null|            []|
|     s1|       v4|          v1|             4|            -9|     0.98|            []|
|     s1|       v4|          v2|

In [21]:
def _neglog_p(p_value_mantissa: Column, p_value_exponent: Column) -> Column:
    """Compute the negative log p-value.

    Args:
        p_value_mantissa (Column): P-value mantissa
        p_value_exponent (Column): P-value exponent

    Returns:
        Column: Negative log p-value

    Examples:
        >>> d = [(1, 1), (5, -2), (1, -1000)]
        >>> df = spark.createDataFrame(d).toDF("p_value_mantissa", "p_value_exponent")
        >>> df.withColumn("neg_log_p", _neglog_p(f.col("p_value_mantissa"), f.col("p_value_exponent"))).show()
        +----------------+----------------+------------------+
        |p_value_mantissa|p_value_exponent|         neg_log_p|
        +----------------+----------------+------------------+
        |               1|               1|              -1.0|
        |               5|              -2|1.3010299956639813|
        |               1|           -1000|            1000.0|
        +----------------+----------------+------------------+
        <BLANKLINE>
    """
    return -1 * (f.log10(p_value_mantissa) + p_value_exponent)


def adding_quality_flag(
    qc_column: Column, flag_condition: Column, flag_text: str
) -> Column:
    """Update the provided quality control list with a new flag if condition is met.

    Args:
        qc_column (Column): Array column with existing QC flags.
        flag_condition (Column): This is a column of booleans, signing which row should be flagged
        flag_text (str): Text for the new quality control flag

    Returns:
        Column: Array column with the updated list of qc flags.

    Examples:
    >>> data = [(True, ['Existing flag']),(True, []),(False, [])]
    >>> new_flag = 'This is a new flag'
    >>> (
    ...     spark.createDataFrame(data, ['flag', 'qualityControl'])
    ...     .withColumn('qualityControl', adding_quality_flag(f.col('qualityControl'), f.col('flag'), new_flag))
    ...     .show(truncate=False)
    ... )
    +-----+-----------------------------------+
    |flag |qualityControl                     |
    +-----+-----------------------------------+
    |true |[Existing flag, This is a new flag]|
    |true |[This is a new flag]               |
    |false|[]                                 |
    +-----+-----------------------------------+
    <BLANKLINE>
    """
    return f.when(
        flag_condition,
        f.array_union(qc_column, f.array(f.lit(flag_text))),
    ).otherwise(qc_column)

w = Window.partitionBy('studyId', 'variantPair').orderBy(f.col('negLogPVal').desc())

(
    df
    # lead/tag pairs irrespective of the order:
    .withColumn('variantPair', f.concat(f.array_sort(f.array(f.col('variantId'), f.col('tagVariantId')))))
    # Generating the negLogPval:
    .withColumn('negLogPVal', _neglog_p(f.col('pValueMantissa'), f.col('pValueExponent')))
    # Counting the number of occurrence for each pair - for one study, each pair should occure only once:
    .withColumn('rank', f.count(f.col('tagVariantId')).over(w))
    # Only the most significant lead is kept if credible sets are overlapping + keeping lead if credible set is not resolved:
    .withColumn('keep_lead', f.when(f.max(f.col('rank')).over(Window.partitionBy('studyId', 'variantId')) <= 1, True).otherwise(False))
    # Adding reference lead that explains:
    .withColumn('reference_lead', f.when(
        f.col('rank') > 1, f.col('tagVariantId')
    ))
    # One credible set might contain multiple tags that are themselves leads. We need to collect them into an array:
    # At this point we move from lead/tag to study/lead level:
    .withColumn(
        'all_explained',
        f.collect_set(f.col('reference_lead')).over(Window.partitionBy('studyId', 'variantId'))
    )
    # For a study we collect all the lead variants that are not explained by other lead:
    .withColumn(
        'good_signals',
        f.collect_set(f.when(f.col('keep_lead'), f.col('variantId'))).over(Window.partitionBy('studyId'))
    )
    # As a lead can be explained by multiple other leads, we select the one, which is not explained by other:
    .withColumn(
        'explained_by',
        f.array_intersect('good_signals', 'all_explained').getItem(0)
    )
    # Dropping credible sets that are explained:
    .filter(f.col('keep_lead') | f.col('reference_lead').isNotNull())
    # Adding QC flag with the respective lead:
    .withColumn(
        'qualityControl', 
        adding_quality_flag(
            f.col('qualityControl'),
            f.col('reference_lead').isNotNull(),
            f.concat_ws(' ', f.lit('Association explained by:'), f.col('explained_by'))
        )
    )
    # Remove tag information if lead is explained by other variant:
    .withColumn(
        'tagVariantId', f.when(f.col('reference_lead').isNull(), f.col('tagVariantId'))
    )
    .withColumn(
        'R_overall', f.when(f.col('reference_lead').isNull(), f.col('R_overall'))
    )
    # Drop unused column:
    .drop('reference_lead', 'variantPair', 'negLogPVal', 'rank', 'keep_lead','all_explained', 'good_signals', 'explained_by')
    .distinct()
    .show(truncate=False)
)

+-------+---------+------------+--------------+--------------+---------+------------------------------+
|studyId|variantId|tagVariantId|pValueMantissa|pValueExponent|R_overall|qualityControl                |
+-------+---------+------------+--------------+--------------+---------+------------------------------+
|s2     |v2       |v1          |1             |-12           |0.98     |[]                            |
|s2     |v2       |v2          |1             |-12           |1.0      |[]                            |
|s1     |v2       |null        |1             |-8            |null     |[Association explained by: v1]|
|s1     |v4       |null        |4             |-9            |null     |[Association explained by: v1]|
|s1     |v3       |null        |1             |-18           |null     |[]                            |
|s1     |v1       |v4          |1             |-9            |0.98     |[]                            |
|s1     |v1       |v1          |1             |-9            |1.

In [1]:
type(spark)

NameError: name 'spark' is not defined

In [1]:
column_names = [
    'studyId',
    'variantId',
    'tagVariantId',
    'pValueMantissa',
    'pValueExponent',
    'R_overall',
]

data = [
    # First study, variant 1 independent association:
    ('s1', 'v1', 'v1', 1, -9, 1.0),
    # First study, variant 3, not resolved in ld:
    ('s1', 'v2', None, 1, -18, None),
    # First study, variant 2, higher p-value, explained by v1:
    ('s1', 'v3', 'v3', 1, -18, 0.98),
    ('s1', 'v3', 'v4', 1, -18, 1.0),
    ('s1', 'v3', 'v5', 1, -18, 0.98),
    # First study, variant 4: higher p-value, explained by v1
    ('s1', 'v4', 'v3', 4, -9, 0.98),
    ('s1', 'v4', 'v5', 4, -9, 0.98),
    ('s1', 'v4', 'v6', 4, -9, 1.0),
    ('s1', 'v4', 'v4', 4, -9, 1.0),
    # Second study, same variant discovered earlier:
    ('s2', 'v5', 'v5', 1, -8, 1.0),
    ('s2', 'v5', 'v3', 1, -8, 1.0),
    ('s2', 'v5', 'v4', 1, -8, 0.98),
    ('s2', 'v5', 'v6', 1, -8, 0.98),
    # Second study, same variant discovered earlier:
    ('s2', 'v6', 'v5', 5, -8, 1.0),
    ('s2', 'v6', 'v4', 5, -8, 1.0),
    ('s2', 'v6', 'v6', 5, -8, 0.98),
]

df = (
    spark.createDataFrame(data, column_names)
    .withColumn('qualityControl', f.array())
    .persist()
)
df.show()

NameError: name 'spark' is not defined